<a href="https://colab.research.google.com/github/sohomghosh/LIPI_ERAI_FinNLP_EMNLP-2022/blob/main/ERAI_SB_1_SB_2_AND_SB_3_SB_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 4.9 MB 18.9 MB/s 
     |████████████████████████████████| 1.3 MB 60.4 MB/s 
     |████████████████████████████████| 120 kB 56.6 MB/s 
     |████████████████████████████████| 6.6 MB 23.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a4007bac8a530fa67069df82387d7738de51b7cd2826b51524d97790c560b501
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer

pd.options.display.float_format = '{:.8f}'.format

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
train_df = pd.read_json("ERAI_Dataset_pairwise_train_with_translation.json").dropna()
train_df

,index,post1,post2,MPP1,MPP2,ML1,ML2,MPP_label,ML_label,post1_en,post2_en
0,0,中壽可以準備賣給開發金了，除權息前應該可以完成,中壽今天發動攻勢，往34靠攏。,0.01063830,0.01204819,-0.00911854,-0.10843373,0,0,Zhongshou can prepare to sell it to the develo...,Zhongshou launched the offensive today and mov...
1,2,"前八月eps 1.94,今天股價還算強勢.",中國人壽今天(19日)下午2時將在證交所發布重大訊息，據了解，中壽將以1元新台幣買下安聯人壽...,0.00507614,0.01567944,-0.04230118,-0.01045296,0,1,"EPS 1.94 the previous August, today's stock pr...",China Life will release a major message at the...
2,4,久違的大漲超過1%。,"再上30,看收盤是否站穩..",0.00000000,0.01010101,-0.06456954,-0.01346801,0,1,The long -lost rose more than 1%.,"Go to 30, see if the closed market is standing..."
3,6,中壽取得安聯人壽約8萬張傳統型保單 有助提升隱含價值\n \n \n 金管會今 (27) 日...,前三月eps 0.97，相當不錯。,0.03050847,0.02254428,-0.00677966,-0.00805153,1,0,"Nakashi obtains about 80,000 traditional insur...","The first March EPS 0.97, quite good."
4,8,"站上32了,再來準備填權息了33.X",1-4月 eps1.11元，今年有機會3元。,0.01774194,0.05882353,-0.00645161,0.00000000,0,1,"Standing on 32, let's prepare for the right to...","From January to April, EPS was 1.11 yuan, and ..."
...,...,...,...,...,...,...,...,...,...,...,...
195,390,看內容說永豐金這個月已追平去年了!!\n \n \n [新聞]永豐金無懼降息環境 前11月累...,永豐金今年最高有到13.5\n 除息完價格是11.65\n 現在股價11.5(還在貼息價)\...,0.00446429,0.00434783,-0.01785714,-0.04347826,1,0,Look at the content saying that Yongfeng Gold ...,Yongfeng Gold is the highest until 13.5 this y...
196,392,永豐金一月EPS 0.15 很不錯!\n \n Ps.永豐去年整年度是1.08,永豐金去年三表現創歷史新高 對台灣今年經濟信心夠\n \n 2021-03-11 經濟日報 ...,0.03097345,0.00404858,0.00000000,-0.02429150,1,0,Yongfeng Gold January EPS 0.15 is very good!\n...,Yongfeng Gold's three performances last year r...
197,394,永豐金現金股利每股0.7元 殖利率衝5.8％\n \n 2021/03/19 工商魏喬怡\n...,永豐金證券2020年複委託交易創下6608億，年成長率達55％，\n 其中複委託零售業務(包...,0.02800000,0.02491103,-0.00400000,-0.00711744,1,0,Yongfeng gold cash dividend 0.7 yuan per share...,Yongfeng Gold Securities commissioned the tran...
198,396,永豐銀加速數位布局，上半年累計基金線上交易比重逾八成\n \n 2021年7月13日\n 【...,《金融股》外資加碼+零股熱絡 永豐金攀高頭\n \n 2021年7月23日【時報記者任珮云台...,0.00349650,0.01060071,-0.01748252,-0.01766784,0,0,Yongfeng Bank accelerates the digital layout.\...,"""Financial Stocks"" foreign capital increase+ze..."


In [6]:
train_data, validation = train_test_split(train_df, test_size=0.2, random_state=42)
#test_gt = pd.read_json("ERAI_Dataset_pairwise_test_GT.json")
#test_gt

In [8]:
train_data_formatted = pd.DataFrame({'index': list(train_data['index']) +list(train_data['index']), 'post_no':[1]*train_data.shape[0]+[2]*train_data.shape[0], 'post_rationale_1_2':list(train_data['post1'])+list(train_data['post2']), 'post_rationale_1_2_english': list(train_data['post1_en'])+list(train_data['post2_en']), 'MPP_1_2':list(train_data['MPP1'])+list(train_data['MPP2']), 'ML_1_2':list(train_data['ML1'])+list(train_data['ML2'])})
train_data_formatted.head(5)

,index,post_no,post_rationale_1_2,post_rationale_1_2_english,MPP_1_2,ML_1_2
0,10,1,水喔\n \n \n 〈中壽法說會〉去年隱含價值年增10.2% 每股EV為64.6元遠高於股...,"Water\n \n \n ""Zhongshou Fa Said"" last year's ...",0.00302572,-0.03479576
1,132,1,友達淨值約$18.16\n \n 10:59的買單拉抬到今日最高價$17.5....國際股市...,AUO net value is about $ 18.16\n \n 10:59 The ...,0.02179837,-0.04087193
2,278,1,"2330已經近二周沒甚麼消息,也許No news is good news,再過一日,202...",There is no news for 2330 for nearly two weeks...,0.07567568,-0.02522523
3,198,1,被這漲勢嚇到不知所措... \n 看來非核家園讓台汽電賺很大,I was scared by this rise ...\n It seems that ...,0.05525847,-0.01247772
4,342,1,漲了10%,10% increased,0.00649351,-0.03246753


In [9]:
validation_formatted = pd.DataFrame({'index': list(validation['index']) +list(validation['index']), 'post_no':[1]*validation.shape[0]+[2]*validation.shape[0], 'post_rationale_1_2':list(validation['post1'])+list(validation['post2']), 'post_rationale_1_2_english': list(validation['post1_en'])+list(validation['post2_en']), 'MPP_1_2':list(validation['MPP1'])+list(validation['MPP2']), 'ML_1_2':list(validation['ML1'])+list(validation['ML2'])})
validation_formatted.head(5)

,index,post_no,post_rationale_1_2,post_rationale_1_2_english,MPP_1_2,ML_1_2
0,284,1,"今天台股2330又將創新高?因為昨晚美股TSM收盤已來到140美元,相當於2330的793元...","Today, Taiwan stocks 2330 will be innovative? ...",0.01435407,-0.03349282
1,230,1,台積電ADR目前價位是114.60\n \n 換算下來是\n (114.60*27.9)/5...,The current price of TSMC ADR is 114.60\n \n C...,0.02113821,-0.00487805
2,32,1,難以置信～～～～～\n \n 我以為這次阿花又勝大盤，想說連續3次打敗大盤，正準備要好好慶祝。,Unbelievable ~~~~~\n \n I thought Ahua was win...,0.00478469,-0.00478469
3,152,1,不認同 不看好 不支持 假存股真價差的\n (勿進勿擾勿留言)\n (2834臺妹)真長期存...,Disagree not to be optimistic that it does not...,0.00480769,-0.00961538
4,314,1,今天除息差強人意，近日外資每日買超一兩百張，慢慢吃貨。,"Today, the division is unsatisfactory. Recentl...",0.00404858,-0.01619433


In [11]:
test_data_1 = pd.read_json("ERAI_Dataset_pairwise_test_with_translation.json")
test_data_1_formatted = pd.DataFrame({'index': list(test_data_1['index']) + list(test_data_1['index']), 'post_no':[1]*test_data_1.shape[0]+[2]*test_data_1.shape[0], 'post_rationale_1_2':list(test_data_1['post1'])+list(test_data_1['post2']), 'post_rationale_1_2_english': list(test_data_1['post1_en'])+list(test_data_1['post2_en'])})
test_data_1_formatted.head(5)

,index,post_no,post_rationale_1_2,post_rationale_1_2_english
0,400,1,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,I walked steadily these two days and felt good...
1,402,1,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,It is said that the unified profit is really s...
2,404,1,統一這一波的漲勢真是猛。\n 有甚麼消息嗎？,The rise of this wave of unification is really...
3,406,1,統一這一波跌得真慘 \n 外資在股價跌到65的時候才降評，\n 是不是真的想要減碼啊？ \n...,The unified wave of this wave has fallen reall...
4,408,1,今天交完稅後，還有一點錢，\n 又進了一張群創，\n 進在11.3，\n 把套牢股票減少一點...,"After paying the tax today, there is still a l..."


In [12]:
unsupervised_test = pd.read_json("ERAI_Dataset_unsupervised_with_translation.json").rename(columns = {"post_rationale":"post_rationale_1_2", "post_rationale_en":"post_rationale_1_2_english"})
unsupervised_test.head(5)

,index,post_rationale_1_2,post_rationale_1_2_english
0,0,關於新潤，很多人都會把新潤建設、新潤興業、華潤給搞混，以為他們是子母公司，事實不 然，雖然...,"\nRegarding Xinrun, many people confuse Xinrun..."
1,1,基本面：這家公司在2012年由大城建設入主，主要建案都在台中，目前公司主要建案有四 筆，另...,\nFundamentals: This company was established b...
2,2,基本介紹: 祺驊股份有限公司成立於1997年三月， 2007年十二月祺驊合併祺友公司，並因...,"\nBasic introduction: Qihua Co., Ltd. was esta..."
3,3,今天盤前發了新聞，導致重挫。 MoneyDJ新聞 20190528 08:45:24 記者...,"\nThere was news before the market today, whic..."
4,4,(文長慎入) 好的，叔叔遵守之前的約定，要來分享這次看好的公司 生平第一次對一家公司的持股...,"\n(Wen Chang enters with caution) Okay, my unc..."


In [13]:
model1 = SentenceTransformer('ProsusAI/finbert')
embedding_train = model1.encode(list(train_data_formatted['post_rationale_1_2_english']))
embedding_valid = model1.encode(list(validation_formatted['post_rationale_1_2_english']))

embedding_test = model1.encode(list(test_data_1_formatted['post_rationale_1_2_english']))
embedding_unsupervised_test = model1.encode(list(unsupervised_test['post_rationale_1_2_english']))

train_data_formatted['finbert_embedding_english'] = [[j for j in i] for i in embedding_train]
validation_formatted['finbert_embedding_english'] = [[j for j in i] for i in embedding_valid]

test_data_1_formatted['finbert_embedding_english'] = [[j for j in i] for i in embedding_test]
unsupervised_test['finbert_embedding_english'] = [[j for j in i] for i in embedding_unsupervised_test]

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
embedding_valid_post1 = model1.encode(list(validation['post1_en']))
embedding_valid_post2 = model1.encode(list(validation['post2_en']))
validation['finbert_embedding_english_post1'] = [[j for j in i] for i in embedding_valid_post1]
validation['finbert_embedding_english_post2'] = [[j for j in i] for i in embedding_valid_post2]

In [17]:
embedding_test_post1 = model1.encode(list(test_data_1['post1_en']))
embedding_test_post2 = model1.encode(list(test_data_1['post2_en']))
test_data_1['finbert_embedding_english_post1'] = [[j for j in i] for i in embedding_test_post1]
test_data_1['finbert_embedding_english_post2'] = [[j for j in i] for i in embedding_test_post2]

In [19]:
model_chinese = SentenceTransformer('DMetaSoul/sbert-chinese-qmc-finance-v1')
embedding_chinese_train = model_chinese.encode(list(train_data_formatted['post_rationale_1_2']))
embedding_chinese_valid = model_chinese.encode(list(validation_formatted['post_rationale_1_2']))

embedding_chinese_test = model1.encode(list(test_data_1_formatted['post_rationale_1_2']))
embedding_chinese_unsupervised_test = model1.encode(list(unsupervised_test['post_rationale_1_2']))

train_data_formatted['dmetasoul_embedding_chinese'] = [[j for j in i] for i in embedding_chinese_train]
validation_formatted['dmetasoul_embedding_chinese'] = [[j for j in i] for i in embedding_chinese_valid]

test_data_1_formatted['dmetasoul_embedding_chinese'] = [[j for j in i] for i in embedding_chinese_test]
unsupervised_test['dmetasoul_embedding_chinese'] = [[j for j in i] for i in embedding_chinese_unsupervised_test]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/835 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/118 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/343 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [20]:
embedding_chinese_valid_post1 = model1.encode(list(validation['post1']))
embedding_chinese_valid_post2 = model1.encode(list(validation['post2']))
validation['dmetasoul_embedding_chinese_post1'] = [[j for j in i] for i in embedding_chinese_valid_post1]
validation['dmetasoul_embedding_chinese_post2'] = [[j for j in i] for i in embedding_chinese_valid_post2]

In [21]:
embedding_chinese_test_post1 = model1.encode(list(test_data_1['post1']))
embedding_chinese_test_post2 = model1.encode(list(test_data_1['post2']))
test_data_1['dmetasoul_embedding_chinese_post1'] = [[j for j in i] for i in embedding_chinese_test_post1]
test_data_1['dmetasoul_embedding_chinese_post2'] = [[j for j in i] for i in embedding_chinese_test_post2]

In [27]:
def mean_absolutePpercentage_error(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred)/y_true))*100

def model_eval(which_set, y_true, y_pred):
  print(which_set)
  print("MAE: ", metrics.mean_absolute_error(y_true, y_pred))
  print("MSE: ", metrics.mean_squared_error(y_true, y_pred))
  print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_true, y_pred)))
  print("MAPE: ", metrics.mean_absolute_percentage_error(y_true, y_pred))
  print("Explained Variance: ", metrics.explained_variance_score(y_true, y_pred))
  print("R2: ", metrics.r2_score(y_true, y_pred))
  print("\n")

In [58]:
def model_eval_predict(clf, train_X, train_y, valid_X, valid_y, test_X, typ, lang, unsuper=False):
  if type(train_X)==pd.core.frame.DataFrame:
    pred_tr_prob = clf.predict(train_X)
    pred_valid_prob = clf.predict(valid_X)
    pred_test_prob = clf.predict(test_X)
  else:
    pred_tr_prob = clf.predict(list(train_X))
    pred_valid_prob = clf.predict(list(valid_X))
    pred_test_prob = clf.predict(list(test_X))
  

  print(clf, typ, lang)

  model_eval("train", train_y, pred_tr_prob)
  model_eval("valid", valid_y, pred_valid_prob)

  train_data_formatted[typ + '_1_2_predicted'] = pred_tr_prob
  train_data_formatted_sorted = train_data_formatted.sort_values(['index', 'post_no']).copy()
  train_data_formatted_sorted_grouped = train_data_formatted_sorted.groupby('index').agg({typ + '_1_2_predicted': lambda x:list(x)}).reset_index()
  if typ=="MPP":
    train_data_formatted_sorted_grouped[typ + '_label_predicted'] = train_data_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 1 if x[0]>x[1] else 0)
  else:
    train_data_formatted_sorted_grouped[typ + '_label_predicted'] = train_data_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 0 if x[0]>x[1] else 1)
  train_data_with_prediction = train_data.merge(train_data_formatted_sorted_grouped, on = "index")
  print(metrics.accuracy_score(train_data_with_prediction[typ + '_label'], train_data_with_prediction[typ + '_label_predicted']))


  validation_formatted[typ + '_1_2_predicted'] = pred_valid_prob
  validation_formatted_sorted = validation_formatted.sort_values(['index', 'post_no']).copy()
  validation_formatted_sorted_grouped = validation_formatted_sorted.groupby('index').agg({typ + '_1_2_predicted': lambda x:list(x)}).reset_index()
  if typ=="MPP":
    validation_formatted_sorted_grouped[typ + '_label_predicted'] = validation_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 1 if x[0]>x[1] else 0)
  else:
    validation_formatted_sorted_grouped[typ + '_label_predicted'] = validation_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 0 if x[0]>x[1] else 1)
  validation_with_prediction = validation.merge(validation_formatted_sorted_grouped, on = "index")
  print(metrics.accuracy_score(validation_with_prediction[typ + '_label'], validation_with_prediction[typ + '_label_predicted']))
  
  if unsuper:
    unsupervised_test[typ] = clf.predict(list(unsupervised_test_X))
    print(list(unsupervised_test.sort_values(typ, ascending = False if typ=="MPP" else True)['index']))

  test_data_1_formatted[typ + '_1_2_predicted'] = pred_test_prob
  test_data_1_formatted_sorted = test_data_1_formatted.sort_values(['index', 'post_no']).copy()
  test_data_1_formatted_sorted_grouped = test_data_1_formatted_sorted.groupby('index').agg({typ + '_1_2_predicted': lambda x:list(x)}).reset_index()
  if typ=="MPP":
    test_data_1_formatted_sorted_grouped[typ + '_label_predicted'] = test_data_1_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 1 if x[0]>x[1] else 0)
  else:
    test_data_1_formatted_sorted_grouped[typ + '_label_predicted'] = test_data_1_formatted_sorted_grouped[typ + '_1_2_predicted'].apply(lambda x : 0 if x[0]>x[1] else 1)
  test_data_1_with_prediction = test_data_1.merge(test_data_1_formatted_sorted_grouped, on = "index")

  return test_data_1_with_prediction


# SB-1 and SB-2

In [30]:
train_X = train_data_formatted.dropna()['finbert_embedding_english']
valid_X = validation_formatted.dropna()['finbert_embedding_english']
test_X = test_data_1_formatted['finbert_embedding_english']
unsupervised_test_X = unsupervised_test['finbert_embedding_english']

clf = LinearRegression(n_jobs=-1)
clf.fit(list(train_X), train_data_formatted.dropna()['MPP_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['MPP_1_2'], valid_X, validation_formatted.dropna()['MPP_1_2'], test_X, "MPP", "Chinese", True)
clf = LinearRegression(n_jobs=-1)
clf.fit(list(train_X), train_data_formatted.dropna()['ML_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['ML_1_2'], valid_X, validation_formatted.dropna()['ML_1_2'], test_X, "ML", "Chinese", True)

clf = MLPRegressor(random_state=1, max_iter=50)
clf.fit(list(train_X), train_data_formatted.dropna()['MPP_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['MPP_1_2'], valid_X, validation_formatted.dropna()['MPP_1_2'], test_X, "MPP", "Chinese", True)
clf = MLPRegressor(random_state=1, max_iter=50)
clf.fit(list(train_X), train_data_formatted.dropna()['ML_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['ML_1_2'], valid_X, validation_formatted.dropna()['ML_1_2'], test_X, "ML", "Chinese", True)


LinearRegression(n_jobs=-1) MPP Chinese
train
MAE:  4.876544693534007e-08
MSE:  3.9599206331791195e-15
RMSE:  6.292790027626156e-08
MAPE:  15486660.92308198
Explained Variance:  0.9999999999934251
R2:  0.999999999993425


valid
MAE:  0.028944299409446463
MSE:  0.0015292370033049845
RMSE:  0.039105460019094325
MAPE:  5964677775362.264
Explained Variance:  -0.5842853907087107
R2:  -0.6847294483185329


1.0
0.375
[201, 205, 167, 168, 169, 38, 148, 208, 156, 22, 151, 31, 174, 207, 68, 19, 144, 155, 90, 12, 200, 177, 159, 6, 91, 189, 70, 195, 96, 139, 37, 206, 162, 8, 16, 110, 131, 50, 60, 5, 204, 145, 20, 119, 182, 65, 175, 198, 92, 141, 97, 191, 64, 114, 62, 45, 128, 34, 104, 161, 72, 118, 11, 146, 199, 173, 15, 154, 99, 158, 197, 190, 39, 138, 85, 54, 2, 209, 74, 115, 67, 80, 183, 89, 93, 63, 18, 88, 203, 25, 106, 153, 135, 28, 111, 57, 112, 129, 123, 24, 73, 179, 164, 105, 125, 21, 55, 41, 132, 40, 53, 120, 134, 108, 23, 100, 150, 29, 52, 86, 126, 79, 3, 59, 75, 122, 109, 143, 113, 17, 

,index,post1,post2,post1_en,post2_en,finbert_embedding_english_post1,finbert_embedding_english_post2,dmetasoul_embedding_chinese_post1,dmetasoul_embedding_chinese_post2,ML_1_2_predicted,ML_label_predicted
0,400,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分...,I walked steadily these two days and felt good...,Shenda: Daizi Silver Star Developments LTD. An...,"[0.6859243, 0.9922623, 0.0006110286, 0.1701935...","[0.20012514, 0.2601195, 0.11118475, 0.06974752...","[0.59779483, 0.54733336, 0.2499663, -0.4233161...","[0.34570265, 0.61518794, 0.22137162, -0.380078...","[0.2636861561784762, -0.03724597896120255]",0
1,402,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,向下探底，今天來到56.5元，以配息兩元，合理股價應該還有一段差距吧！讓我們再看下去,It is said that the unified profit is really s...,"Fix the bottom, come to 56.5 yuan today, with ...","[0.30520272, 0.24957183, -0.061990555, 0.18187...","[-0.018197378, 0.17821094, -0.006141174, 0.156...","[0.41023898, 0.5792839, 0.35369316, -0.6365166...","[0.32343695, 0.3703615, 0.24363871, -0.7664103...","[-0.10304961177493799, -0.0767076355407153]",1
2,404,統一這一波的漲勢真是猛。\n 有甚麼消息嗎？,統一的股價創新高...\n 對我這個長期定存統一股票的人，\n 只能說,The rise of this wave of unification is really...,Unified stock price innovation high ...\n For ...,"[0.3131802, -0.34124872, 0.055987928, -0.05064...","[0.25530535, 0.3128814, 0.21723863, -0.2086214...","[0.27578625, 0.50437915, -0.13994099, -0.73864...","[0.2940179, 0.68503374, -0.076299615, -0.69605...","[-0.22004318823459829, 0.00816657969575832]",1
3,406,統一這一波跌得真慘 \n 外資在股價跌到65的時候才降評，\n 是不是真的想要減碼啊？ \n...,盤中給賣掉，現在給我漲起~~氣死人,The unified wave of this wave has fallen reall...,"Sold it on the plate, and now give me up ~~ Fu...","[-0.15165314, 0.39070415, 0.10553888, -0.05212...","[0.32999742, 0.0005482225, 0.020661937, -0.088...","[0.2707702, 0.81925255, 0.43653572, -0.4594372...","[-0.1735457, 0.14491677, 0.41937453, -0.666660...","[-0.15821095988995493, 0.0566880668376078]",1
4,408,今天交完稅後，還有一點錢，\n 又進了一張群創，\n 進在11.3，\n 把套牢股票減少一點...,融資昨天暴增 要小心點\n 可能又要盤個幾天才會繼續往上攻\n 27日資券尚未出來~ 繼續看...,"After paying the tax today, there is still a l...",Funding is so careful yesterday\n Maybe it may...,"[-0.0681871, 0.16403544, -0.039320927, 0.04524...","[0.37033996, 0.000959645, 0.31891954, 0.401414...","[0.44620395, 0.5028211, 0.15858072, -0.7689333...","[0.2347061, 0.5476678, -0.02444358, -0.8017170...","[-0.006195644023430286, -0.015728999301763635]",0
...,...,...,...,...,...,...,...,...,...,...,...
82,564,昨天手真的太癢\n 掛跌停小買一張\n 竟然有買到\n \n 爽啦,長榮沒意外的話，週四要被關了\n 陽明接下來有機會超車,Yesterday it was really itchy\n Buy one in the...,"If Evergreen is not surprised, it will be clos...","[0.09891961, 0.03715299, -0.048978977, 0.07355...","[0.037736874, -0.27821398, 0.2481085, 0.019750...","[0.38141084, 0.6220578, -0.30403435, -0.665752...","[0.32039815, 0.6126691, 0.068923764, -0.711237...","[-0.04741390284426064, 0.02869403971733847]",1
83,566,今天早盤拉上去的時候\n 想不開\n 就跳海了\n 剩下的給別人賺\n 祝大家多空都賺大錢\...,今天還能拉漲停的話\n \n 那他真的是海賊王了\n \n 哈哈哈!!!!\n \n 不可能...,"When I pulled up this morning, I went up early...",If you can pull the daily limit today\n \n The...,"[0.22485298, 0.40419072, 0.046977628, -0.02726...","[0.11620488, 0.4570548, 0.08846752, -0.3904846...","[0.046648428, 0.84257126, 0.14790465, -0.71993...","[0.096996605, 0.68262774, 0.0058463304, -1.150...","[0.0044945374192386645, -0.11392379367787925]",0
84,568,"不知各位對於這隻股的看法如何?? 是在2010/11 Money 雜誌上看到推薦這隻股,看他...",近期外資回補了八萬多張，\n 看起來後勢可期^^,I wonder what the opinion of this stock ?? I s...,"Recently, foreign capital has replenished more...","[0.13683803, 0.052711, 0.11344689, 0.18187611,...","[0.126638, 0.1757161, 0.35074553, -0.001050096...","[0.089465216, 0.9457052, 0.14676356, -0.494651...","[0.3613182, 0.6352121, 0.22215028, -0.67816705...","[-0.07927225842628541, -0.18561660801666874]",0
85,570,BDI指數持續看漲，前10大航商組成三大聯盟，佔全球運力近八成，談價空間有利於航商，目前雞蛋...,我朋友於去年4月跟我對賭\n 說一年內鴻海的股價一定會落到40 (只要有碰到40就算)\n ...,The BDI index continues to rise. The top 10 na...,My friend gamble

In [31]:
train_X = train_data_formatted.dropna()['dmetasoul_embedding_chinese']
valid_X = validation_formatted.dropna()['dmetasoul_embedding_chinese']
test_X = test_data_1_formatted['dmetasoul_embedding_chinese']
unsupervised_test_X = unsupervised_test['dmetasoul_embedding_chinese']

clf = LinearRegression(n_jobs=-1)
clf.fit(list(train_X), train_data_formatted.dropna()['MPP_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['MPP_1_2'], valid_X, validation_formatted.dropna()['MPP_1_2'], test_X, "MPP", "Chinese", True)
clf = LinearRegression(n_jobs=-1)
clf.fit(list(train_X), train_data_formatted.dropna()['MPP_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['ML_1_2'], valid_X, validation_formatted.dropna()['ML_1_2'], test_X, "ML", "Chinese", True)

clf = MLPRegressor(random_state=1, max_iter=50)
clf.fit(list(train_X), train_data_formatted.dropna()['ML_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['MPP_1_2'], valid_X, validation_formatted.dropna()['MPP_1_2'], test_X, "MPP", "Chinese", True)
clf =  MLPRegressor(random_state=1, max_iter=50)
clf.fit(list(train_X), train_data_formatted.dropna()['ML_1_2'])
model_eval_predict(clf, train_X, train_data_formatted.dropna()['ML_1_2'], valid_X, validation_formatted.dropna()['ML_1_2'], test_X, "ML", "Chinese", True)

LinearRegression(n_jobs=-1) MPP Chinese
train
MAE:  4.591141043535606e-08
MSE:  3.291074343245937e-15
RMSE:  5.7367885992477856e-08
MAPE:  14901878.153851131
Explained Variance:  0.999999999994583
R2:  0.9999999999945356


valid
MAE:  0.033418754123820796
MSE:  0.001706197865689527
RMSE:  0.04130614803742328
MAPE:  10526072504322.498
Explained Variance:  -0.8694087768547569
R2:  -0.8796836479715373


0.9935897435897436
0.7
[50, 138, 39, 166, 47, 44, 175, 191, 13, 54, 46, 57, 48, 97, 6, 141, 33, 140, 187, 60, 190, 192, 100, 29, 150, 18, 88, 129, 63, 99, 15, 154, 58, 188, 161, 112, 67, 115, 74, 69, 65, 10, 136, 183, 37, 96, 139, 86, 126, 5, 52, 130, 32, 4, 124, 82, 56, 104, 128, 45, 34, 78, 87, 3, 149, 41, 80, 27, 117, 147, 62, 207, 36, 137, 194, 89, 123, 73, 24, 98, 95, 171, 75, 122, 9, 109, 91, 159, 8, 186, 155, 90, 12, 71, 134, 23, 108, 76, 64, 94, 208, 53, 66, 200, 68, 172, 103, 93, 204, 21, 55, 125, 196, 205, 79, 162, 164, 163, 83, 142, 0, 77, 121, 35, 132, 40, 70, 38, 148, 22, 156,

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPRegressor(max_iter=50, random_state=1) MPP Chinese
train
MAE:  0.06659121218772292
MSE:  0.006954580901320944
RMSE:  0.08339412989725922
MAPE:  24630178080690.543
Explained Variance:  -6.7869555367839265
R2:  -10.547152135648451


valid
MAE:  0.14863554284169803
MSE:  0.038745338030078416
RMSE:  0.1968383550786747
MAPE:  25527263610419.45
Explained Variance:  -38.11778560720643
R2:  -41.68495453828018


0.5641025641025641
0.65
[173, 165, 174, 193, 206, 187, 49, 157, 26, 116, 179, 71, 170, 47, 72, 53, 164, 185, 119, 190, 76, 202, 199, 182, 145, 20, 43, 111, 135, 28, 113, 143, 17, 152, 102, 7, 186, 122, 109, 9, 75, 93, 162, 73, 24, 123, 131, 110, 16, 89, 115, 74, 40, 132, 198, 184, 112, 181, 183, 204, 58, 65, 104, 128, 34, 45, 203, 81, 86, 126, 5, 52, 106, 25, 153, 11, 118, 146, 44, 207, 201, 61, 156, 22, 148, 38, 80, 62, 64, 98, 120, 144, 19, 160, 1, 84, 151, 31, 191, 108, 134, 23, 175, 103, 176, 121, 35, 77, 166, 200, 78, 192, 3, 87, 149, 92, 197, 178, 139, 37, 96, 177, 57, 188, 97,

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPRegressor(max_iter=50, random_state=1) ML Chinese
train
MAE:  0.046907503004324023
MSE:  0.0038790973074407783
RMSE:  0.0622823996602634
MAPE:  19187887491088.07
Explained Variance:  -2.731485937231151
R2:  -2.7320309630943176


valid
MAE:  0.13913156212192096
MSE:  0.03461980815385464
RMSE:  0.18606398940647983
MAPE:  68367650393383.086
Explained Variance:  -56.48216977303783
R2:  -56.70243906326978


0.5769230769230769
0.55
[209, 46, 172, 169, 168, 167, 140, 33, 163, 195, 48, 66, 59, 41, 50, 99, 15, 154, 138, 39, 70, 130, 32, 79, 114, 83, 142, 0, 117, 27, 147, 124, 82, 56, 4, 18, 129, 88, 60, 150, 29, 100, 8, 91, 159, 125, 55, 21, 205, 171, 196, 94, 208, 95, 63, 194, 42, 13, 180, 69, 107, 14, 133, 161, 189, 155, 90, 12, 68, 137, 36, 2, 85, 158, 105, 54, 101, 30, 127, 51, 67, 10, 136, 141, 6, 97, 188, 57, 177, 96, 139, 37, 178, 197, 92, 3, 87, 149, 192, 78, 200, 166, 121, 77, 35, 176, 103, 175, 134, 23, 108, 191, 151, 31, 84, 160, 1, 144, 19, 120, 98, 64, 62, 80, 38, 148, 156, 22, 

,index,post1,post2,post1_en,post2_en,finbert_embedding_english_post1,finbert_embedding_english_post2,dmetasoul_embedding_chinese_post1,dmetasoul_embedding_chinese_post2,ML_1_2_predicted,ML_label_predicted
0,400,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分...,I walked steadily these two days and felt good...,Shenda: Daizi Silver Star Developments LTD. An...,"[0.6859243, 0.9922623, 0.0006110286, 0.1701935...","[0.20012514, 0.2601195, 0.11118475, 0.06974752...","[0.59779483, 0.54733336, 0.2499663, -0.4233161...","[0.34570265, 0.61518794, 0.22137162, -0.380078...","[-0.4649937183726305, -0.413955581681274]",1
1,402,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,向下探底，今天來到56.5元，以配息兩元，合理股價應該還有一段差距吧！讓我們再看下去,It is said that the unified profit is really s...,"Fix the bottom, come to 56.5 yuan today, with ...","[0.30520272, 0.24957183, -0.061990555, 0.18187...","[-0.018197378, 0.17821094, -0.006141174, 0.156...","[0.41023898, 0.5792839, 0.35369316, -0.6365166...","[0.32343695, 0.3703615, 0.24363871, -0.7664103...","[-0.4329135294014029, -0.5427078496517117]",0
2,404,統一這一波的漲勢真是猛。\n 有甚麼消息嗎？,統一的股價創新高...\n 對我這個長期定存統一股票的人，\n 只能說,The rise of this wave of unification is really...,Unified stock price innovation high ...\n For ...,"[0.3131802, -0.34124872, 0.055987928, -0.05064...","[0.25530535, 0.3128814, 0.21723863, -0.2086214...","[0.27578625, 0.50437915, -0.13994099, -0.73864...","[0.2940179, 0.68503374, -0.076299615, -0.69605...","[-0.5063107921345003, -0.6182329646123133]",0
3,406,統一這一波跌得真慘 \n 外資在股價跌到65的時候才降評，\n 是不是真的想要減碼啊？ \n...,盤中給賣掉，現在給我漲起~~氣死人,The unified wave of this wave has fallen reall...,"Sold it on the plate, and now give me up ~~ Fu...","[-0.15165314, 0.39070415, 0.10553888, -0.05212...","[0.32999742, 0.0005482225, 0.020661937, -0.088...","[0.2707702, 0.81925255, 0.43653572, -0.4594372...","[-0.1735457, 0.14491677, 0.41937453, -0.666660...","[-0.594898842431791, -0.6025063409612155]",0
4,408,今天交完稅後，還有一點錢，\n 又進了一張群創，\n 進在11.3，\n 把套牢股票減少一點...,融資昨天暴增 要小心點\n 可能又要盤個幾天才會繼續往上攻\n 27日資券尚未出來~ 繼續看...,"After paying the tax today, there is still a l...",Funding is so careful yesterday\n Maybe it may...,"[-0.0681871, 0.16403544, -0.039320927, 0.04524...","[0.37033996, 0.000959645, 0.31891954, 0.401414...","[0.44620395, 0.5028211, 0.15858072, -0.7689333...","[0.2347061, 0.5476678, -0.02444358, -0.8017170...","[-0.7147013827251136, -0.7033260387317715]",1
...,...,...,...,...,...,...,...,...,...,...,...
82,564,昨天手真的太癢\n 掛跌停小買一張\n 竟然有買到\n \n 爽啦,長榮沒意外的話，週四要被關了\n 陽明接下來有機會超車,Yesterday it was really itchy\n Buy one in the...,"If Evergreen is not surprised, it will be clos...","[0.09891961, 0.03715299, -0.048978977, 0.07355...","[0.037736874, -0.27821398, 0.2481085, 0.019750...","[0.38141084, 0.6220578, -0.30403435, -0.665752...","[0.32039815, 0.6126691, 0.068923764, -0.711237...","[-0.3120314191034871, -0.5305988737500078]",0
83,566,今天早盤拉上去的時候\n 想不開\n 就跳海了\n 剩下的給別人賺\n 祝大家多空都賺大錢\...,今天還能拉漲停的話\n \n 那他真的是海賊王了\n \n 哈哈哈!!!!\n \n 不可能...,"When I pulled up this morning, I went up early...",If you can pull the daily limit today\n \n The...,"[0.22485298, 0.40419072, 0.046977628, -0.02726...","[0.11620488, 0.4570548, 0.08846752, -0.3904846...","[0.046648428, 0.84257126, 0.14790465, -0.71993...","[0.096996605, 0.68262774, 0.0058463304, -1.150...","[-0.6399678131438837, -0.5480329274397993]",1
84,568,"不知各位對於這隻股的看法如何?? 是在2010/11 Money 雜誌上看到推薦這隻股,看他...",近期外資回補了八萬多張，\n 看起來後勢可期^^,I wonder what the opinion of this stock ?? I s...,"Recently, foreign capital has replenished more...","[0.13683803, 0.052711, 0.11344689, 0.18187611,...","[0.126638, 0.1757161, 0.35074553, -0.001050096...","[0.089465216, 0.9457052, 0.14676356, -0.494651...","[0.3613182, 0.6352121, 0.22215028, -0.67816705...","[-0.6985840039036038, -0.3719947619875691]",1
85,570,BDI指數持續看漲，前10大航商組成三大聯盟，佔全球運力近八成，談價空間有利於航商，目前雞蛋...,我朋友於去年4月跟我對賭\n 說一年內鴻海的股價一定會落到40 (只要有碰到40就算)\n ...,The BDI index continues to rise. The top 10 na...,My friend gambled me in April 

# SB-3 and SB-5

In [66]:
def model_eval_predict_n2(clf, train_X, train_y, valid_X, valid_y, test_X, typ, lang, unsuper=False):
  if type(train_X)==pd.core.frame.DataFrame:
    pred_tr_prob = clf.predict(train_X)
    pred_valid_prob = clf.predict(valid_X)
    pred_test_prob = clf.predict(test_X)
  else:
    pred_tr_prob = clf.predict(list(train_X))
    pred_valid_prob = clf.predict(list(valid_X))
    pred_test_prob = clf.predict(list(test_X))
  

  print(clf, typ, lang)

  model_eval("train", train_y, pred_tr_prob)
  model_eval("valid", valid_y, pred_valid_prob)

  train_data_formatted_cross_non_dup[typ + '_1_2_predicted'] = pred_tr_prob
  
 
  if typ=="MPP":
    train_data_formatted_cross_non_dup[typ + '_label_predicted'] = train_data_formatted_cross_non_dup[typ + '_1_2_predicted'].apply(lambda x : 1 if x>0 else 0)
  else:
    train_data_formatted_cross_non_dup[typ + '_label_predicted'] = train_data_formatted_cross_non_dup[typ + '_1_2_predicted'].apply(lambda x : 0 if x>0 else 1)
 
  print(metrics.accuracy_score(train_data_formatted_cross_non_dup[typ + '_label'], train_data_formatted_cross_non_dup[typ + '_label_predicted']))


  validation[typ + '_1_2_predicted'] = pred_valid_prob
  if typ=="MPP":
    validation[typ + '_label_predicted'] = validation[typ + '_1_2_predicted'].apply(lambda x : 1 if x>0 else 0)
  else:
    validation[typ + '_label_predicted'] = validation[typ + '_1_2_predicted'].apply(lambda x : 0 if x>0 else 1)
  print(metrics.accuracy_score(validation[typ + '_label'], validation[typ + '_label_predicted']))
  
  if unsuper:
    unsupervised_test[typ] = clf.predict(list(unsupervised_test_X))
    print(list(unsupervised_test.sort_values(typ, ascending = False if typ=="MPP" else True)['index']))

  test_data_1[typ + '_1_2_predicted'] = pred_test_prob
  if typ=="MPP":
    test_data_1[typ + '_label_predicted'] = test_data_1[typ + '_1_2_predicted'].apply(lambda x : 1 if x>0 else 0)
  else:
    test_data_1[typ + '_label_predicted'] = test_data_1[typ + '_1_2_predicted'].apply(lambda x : 0 if x>0 else 1)

  return test_data_1


In [35]:
train_data_formatted['key'] = 0
train_data_formatted_cross = train_data_formatted.merge(train_data_formatted, on = "key").drop("key", 1)
train_data_formatted_cross_non_dup = train_data_formatted_cross[(train_data_formatted_cross['index_x']!=train_data_formatted_cross['index_y']) | (train_data_formatted_cross['post_no_x']!=train_data_formatted_cross['post_no_y'])].copy()

train_data_formatted_cross_non_dup['MPP1_MPP2_diff'] = train_data_formatted_cross_non_dup['MPP_1_2_x'] - train_data_formatted_cross_non_dup['MPP_1_2_y']
train_data_formatted_cross_non_dup['ML1_ML2_diff'] = train_data_formatted_cross_non_dup['ML_1_2_x'] - train_data_formatted_cross_non_dup['ML_1_2_y']

train_data_formatted_cross_non_dup['MPP1_MPP2_diff'] = train_data_formatted_cross_non_dup['MPP_1_2_x'] - train_data_formatted_cross_non_dup['MPP_1_2_y']
train_data_formatted_cross_non_dup['ML1_ML2_diff'] = train_data_formatted_cross_non_dup['ML_1_2_x'] - train_data_formatted_cross_non_dup['ML_1_2_y']

validation['MPP1_MPP2_diff'] = validation['MPP1'] - validation['MPP2']
validation['ML1_ML2_diff'] = validation['ML1'] - validation['ML2']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [64]:
train_data_formatted_cross_non_dup['MPP_label'] = train_data_formatted_cross_non_dup.apply(lambda x : 1 if x.MPP_1_2_x > x.MPP_1_2_y else 0, axis =1) 
train_data_formatted_cross_non_dup["ML_label"] = train_data_formatted_cross_non_dup.apply(lambda x : 0 if x.ML_1_2_x > x.ML_1_2_y else 1, axis =1) 

In [69]:
train_X = pd.concat([pd.DataFrame(list(train_data_formatted_cross_non_dup['finbert_embedding_english_x'])), pd.DataFrame(list(train_data_formatted_cross_non_dup['finbert_embedding_english_y']))], axis = 1)
valid_X = pd.concat([pd.DataFrame(list(validation['finbert_embedding_english_post1'])), pd.DataFrame(list(validation['finbert_embedding_english_post2']))], axis = 1)
test_X = pd.concat([pd.DataFrame(list(test_data_1['finbert_embedding_english_post1'])), pd.DataFrame(list(test_data_1['finbert_embedding_english_post2']))], axis = 1)
unsupervised_test_X = unsupervised_test['finbert_embedding_english']

clf = LinearRegression(n_jobs=-1)
clf.fit(train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'], valid_X, validation['MPP1_MPP2_diff'], test_X, "MPP", "English")
clf = LinearRegression(n_jobs=-1)
clf.fit(train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'], valid_X, validation['ML1_ML2_diff'], test_X, "ML", "English")

clf = MLPRegressor(random_state=1, max_iter=500)
clf.fit(train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'], valid_X, validation['MPP1_MPP2_diff'], test_X, "MPP", "English")
clf = MLPRegressor(random_state=1, max_iter=500)
clf.fit(train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'], valid_X, validation['ML1_ML2_diff'], test_X, "ML", "English")


LinearRegression(n_jobs=-1) MPP English
train
MAE:  1.8585566155694585e-07
MSE:  5.510882971949498e-14
RMSE:  2.3475269906753995e-07
MAPE:  6024818.898824579
Explained Variance:  0.9999999999546583
R2:  0.9999999999543963


valid
MAE:  0.04648493419145815
MSE:  0.0034159513861996026
RMSE:  0.0584461409008294
MAPE:  43.55327186043186
Explained Variance:  -0.9493494741130342
R2:  -1.0252261299131118


0.9954860252287905
0.4
LinearRegression(n_jobs=-1) ML English
train
MAE:  2.263670194422118e-07
MSE:  8.365310451156913e-14
RMSE:  2.892284642139655e-07
MAPE:  5326448.617647936
Explained Variance:  0.9999999999601843
R2:  0.9999999999598882


valid
MAE:  0.04887927157118989
MSE:  0.0035781702461464117
RMSE:  0.05981780877085362
MAPE:  41.450474889172796
Explained Variance:  -1.5349721871159812
R2:  -1.5400007389516328


0.9965990601038832
0.525
MLPRegressor(max_iter=500, random_state=1) MPP English
train
MAE:  0.003404296846033881
MSE:  1.4974616708501323e-05
RMSE:  0.003869704989854049
MA

,index,post1,post2,post1_en,post2_en,finbert_embedding_english_post1,finbert_embedding_english_post2,dmetasoul_embedding_chinese_post1,dmetasoul_embedding_chinese_post2,MPP_1_2_predicted,MPP_label_predicted,ML_1_2_predicted,ML_label_predicted
0,400,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分...,I walked steadily these two days and felt good...,Shenda: Daizi Silver Star Developments LTD. An...,"[0.6859243, 0.9922623, 0.0006110286, 0.1701935...","[0.20012514, 0.2601195, 0.11118475, 0.06974752...","[0.59779483, 0.54733336, 0.2499663, -0.4233161...","[0.34570265, 0.61518794, 0.22137162, -0.380078...",-0.06303637,0,-0.07934363,1
1,402,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,向下探底，今天來到56.5元，以配息兩元，合理股價應該還有一段差距吧！讓我們再看下去,It is said that the unified profit is really s...,"Fix the bottom, come to 56.5 yuan today, with ...","[0.30520272, 0.24957183, -0.061990555, 0.18187...","[-0.018197378, 0.17821094, -0.006141174, 0.156...","[0.41023898, 0.5792839, 0.35369316, -0.6365166...","[0.32343695, 0.3703615, 0.24363871, -0.7664103...",-0.04306892,0,-0.00440014,1
2,404,統一這一波的漲勢真是猛。\n 有甚麼消息嗎？,統一的股價創新高...\n 對我這個長期定存統一股票的人，\n 只能說,The rise of this wave of unification is really...,Unified stock price innovation high ...\n For ...,"[0.3131802, -0.34124872, 0.055987928, -0.05064...","[0.25530535, 0.3128814, 0.21723863, -0.2086214...","[0.27578625, 0.50437915, -0.13994099, -0.73864...","[0.2940179, 0.68503374, -0.076299615, -0.69605...",0.03350371,1,-0.09488264,1
3,406,統一這一波跌得真慘 \n 外資在股價跌到65的時候才降評，\n 是不是真的想要減碼啊？ \n...,盤中給賣掉，現在給我漲起~~氣死人,The unified wave of this wave has fallen reall...,"Sold it on the plate, and now give me up ~~ Fu...","[-0.15165314, 0.39070415, 0.10553888, -0.05212...","[0.32999742, 0.0005482225, 0.020661937, -0.088...","[0.2707702, 0.81925255, 0.43653572, -0.4594372...","[-0.1735457, 0.14491677, 0.41937453, -0.666660...",-0.02786153,0,-0.13483271,1
4,408,今天交完稅後，還有一點錢，\n 又進了一張群創，\n 進在11.3，\n 把套牢股票減少一點...,融資昨天暴增 要小心點\n 可能又要盤個幾天才會繼續往上攻\n 27日資券尚未出來~ 繼續看...,"After paying the tax today, there is still a l...",Funding is so careful yesterday\n Maybe it may...,"[-0.0681871, 0.16403544, -0.039320927, 0.04524...","[0.37033996, 0.000959645, 0.31891954, 0.401414...","[0.44620395, 0.5028211, 0.15858072, -0.7689333...","[0.2347061, 0.5476678, -0.02444358, -0.8017170...",-0.00508677,0,-0.01559420,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,564,昨天手真的太癢\n 掛跌停小買一張\n 竟然有買到\n \n 爽啦,長榮沒意外的話，週四要被關了\n 陽明接下來有機會超車,Yesterday it was really itchy\n Buy one in the...,"If Evergreen is not surprised, it will be clos...","[0.09891961, 0.03715299, -0.048978977, 0.07355...","[0.037736874, -0.27821398, 0.2481085, 0.019750...","[0.38141084, 0.6220578, -0.30403435, -0.665752...","[0.32039815, 0.6126691, 0.068923764, -0.711237...",0.05101725,1,-0.02203593,1
83,566,今天早盤拉上去的時候\n 想不開\n 就跳海了\n 剩下的給別人賺\n 祝大家多空都賺大錢\...,今天還能拉漲停的話\n \n 那他真的是海賊王了\n \n 哈哈哈!!!!\n \n 不可能...,"When I pulled up this morning, I went up early...",If you can pull the daily limit today\n \n The...,"[0.22485298, 0.40419072, 0.046977628, -0.02726...","[0.11620488, 0.4570548, 0.08846752, -0.3904846...","[0.046648428, 0.84257126, 0.14790465, -0.71993...","[0.096996605, 0.68262774, 0.0058463304, -1.150...",-0.00281129,0,-0.18910909,1
84,568,"不知各位對於這隻股的看法如何?? 是在2010/11 Money 雜誌上看到推薦這隻股,看他...",近期外資回補了八萬多張，\n 看起來後勢可期^^,I wonder what the opinion of this stock ?? I s...,"Recently, foreign capital has replenished more...","[0.13683803, 0.052711, 0.11344689, 0.18187611,...","[0.126638, 0.1757161, 0.35074553, -0.001050096...","[0.089465216, 0.9457052, 0.14676356, -0.494651...","[0.3613182, 0.6352121, 0.22215028, -0.67816705...",-0.06603495,0,-0.05260485,1
85,570,BDI指數持續看漲，前10大航商組成三大聯盟，佔全球運力近八成，談價空間有利於航商，目前雞蛋...,我朋友於去年4月跟我對賭\n 說一年內鴻海的股價一定會落到40 (只要有碰到40就算)\n ...,The BDI index continues to rise. The top 10 na...,My friend gambled me in April last year\n It i...,"[0.195929, 0.4238964, 0.107287586, 0.13004434,...","[-0.15792839, 0.18903646, 0.36343

In [73]:
train_X = pd.concat([pd.DataFrame(list(train_data_formatted_cross_non_dup['dmetasoul_embedding_chinese_x'])), pd.DataFrame(list(train_data_formatted_cross_non_dup['dmetasoul_embedding_chinese_y']))], axis = 1)
valid_X = pd.concat([pd.DataFrame(list(validation['dmetasoul_embedding_chinese_post1'])), pd.DataFrame(list(validation['dmetasoul_embedding_chinese_post2']))], axis = 1)
test_X = pd.concat([pd.DataFrame(list(test_data_1['dmetasoul_embedding_chinese_post1'])), pd.DataFrame(list(test_data_1['dmetasoul_embedding_chinese_post2']))], axis = 1)
unsupervised_test_X = unsupervised_test['dmetasoul_embedding_chinese']

clf = LinearRegression(n_jobs=-1)
clf.fit(train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'], valid_X, validation['MPP1_MPP2_diff'], test_X, "MPP", "Chinese")
clf = LinearRegression(n_jobs=-1)
clf.fit(train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'], valid_X, validation['ML1_ML2_diff'], test_X, "ML", "Chinese")

clf = MLPRegressor(random_state=1, max_iter=500)
clf.fit(train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['MPP1_MPP2_diff'], valid_X, validation['MPP1_MPP2_diff'], test_X, "MPP", "Chinese")
clf = MLPRegressor(random_state=1, max_iter=500)
clf.fit(train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'])
model_eval_predict_n2(clf, train_X, train_data_formatted_cross_non_dup['ML1_ML2_diff'], valid_X, validation['ML1_ML2_diff'], test_X, "ML", "Chinese")


LinearRegression(n_jobs=-1) MPP Chinese
train
MAE:  1.245134245621481e-07
MSE:  2.446393878373964e-14
RMSE:  1.5640952267601752e-07
MAPE:  3707771.4985071635
Explained Variance:  0.9999999999797572
R2:  0.9999999999797555


valid
MAE:  0.09334688540175562
MSE:  0.010338067734436834
RMSE:  0.10167628894898177
MAPE:  107.50078714040589
Explained Variance:  -0.3915206786572647
R2:  -5.129163603784883


0.9963929425344217
0.45
LinearRegression(n_jobs=-1) ML Chinese
train
MAE:  3.4386689999338536e-07
MSE:  1.8544080760341955e-13
RMSE:  4.3062838689921447e-07
MAPE:  11643026.076832974
Explained Variance:  0.9999999999111702
R2:  0.9999999999110808


valid
MAE:  0.045092170390725474
MSE:  0.003620848327206967
RMSE:  0.060173485250623195
MAPE:  40.19769083938693
Explained Variance:  -1.5700230411162055
R2:  -1.5702962112108398


0.9977224008574491
0.475
MLPRegressor(max_iter=500, random_state=1) MPP Chinese
train
MAE:  0.011741160267441018
MSE:  0.00014716403308344443
RMSE:  0.0121311183772743

,index,post1,post2,post1_en,post2_en,finbert_embedding_english_post1,finbert_embedding_english_post2,dmetasoul_embedding_chinese_post1,dmetasoul_embedding_chinese_post2,MPP_1_2_predicted,MPP_label_predicted,ML_1_2_predicted,ML_label_predicted
0,400,這一兩天穩穩地走，感覺不錯！\n 雖然失去攤平的機會，但是虧損穩定變少，也可以多留點現金在身...,神達:代子公司Silver Star Developments Ltd.公告董事會決議股利分...,I walked steadily these two days and felt good...,Shenda: Daizi Silver Star Developments LTD. An...,"[0.6859243, 0.9922623, 0.0006110286, 0.1701935...","[0.20012514, 0.2601195, 0.11118475, 0.06974752...","[0.59779483, 0.54733336, 0.2499663, -0.4233161...","[0.34570265, 0.61518794, 0.22137162, -0.380078...",0.61964381,1,0.59654486,0
1,402,話說統一獲利真的很穩定，每股盈餘年年都差不多2塊半左右，以前還有配股，今年公告僅配息2元，配...,向下探底，今天來到56.5元，以配息兩元，合理股價應該還有一段差距吧！讓我們再看下去,It is said that the unified profit is really s...,"Fix the bottom, come to 56.5 yuan today, with ...","[0.30520272, 0.24957183, -0.061990555, 0.18187...","[-0.018197378, 0.17821094, -0.006141174, 0.156...","[0.41023898, 0.5792839, 0.35369316, -0.6365166...","[0.32343695, 0.3703615, 0.24363871, -0.7664103...",0.42147964,1,0.41221309,0
2,404,統一這一波的漲勢真是猛。\n 有甚麼消息嗎？,統一的股價創新高...\n 對我這個長期定存統一股票的人，\n 只能說,The rise of this wave of unification is really...,Unified stock price innovation high ...\n For ...,"[0.3131802, -0.34124872, 0.055987928, -0.05064...","[0.25530535, 0.3128814, 0.21723863, -0.2086214...","[0.27578625, 0.50437915, -0.13994099, -0.73864...","[0.2940179, 0.68503374, -0.076299615, -0.69605...",0.47348663,1,0.47453880,0
3,406,統一這一波跌得真慘 \n 外資在股價跌到65的時候才降評，\n 是不是真的想要減碼啊？ \n...,盤中給賣掉，現在給我漲起~~氣死人,The unified wave of this wave has fallen reall...,"Sold it on the plate, and now give me up ~~ Fu...","[-0.15165314, 0.39070415, 0.10553888, -0.05212...","[0.32999742, 0.0005482225, 0.020661937, -0.088...","[0.2707702, 0.81925255, 0.43653572, -0.4594372...","[-0.1735457, 0.14491677, 0.41937453, -0.666660...",0.70096040,1,0.70793319,0
4,408,今天交完稅後，還有一點錢，\n 又進了一張群創，\n 進在11.3，\n 把套牢股票減少一點...,融資昨天暴增 要小心點\n 可能又要盤個幾天才會繼續往上攻\n 27日資券尚未出來~ 繼續看...,"After paying the tax today, there is still a l...",Funding is so careful yesterday\n Maybe it may...,"[-0.0681871, 0.16403544, -0.039320927, 0.04524...","[0.37033996, 0.000959645, 0.31891954, 0.401414...","[0.44620395, 0.5028211, 0.15858072, -0.7689333...","[0.2347061, 0.5476678, -0.02444358, -0.8017170...",0.58350801,1,0.56436926,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,564,昨天手真的太癢\n 掛跌停小買一張\n 竟然有買到\n \n 爽啦,長榮沒意外的話，週四要被關了\n 陽明接下來有機會超車,Yesterday it was really itchy\n Buy one in the...,"If Evergreen is not surprised, it will be clos...","[0.09891961, 0.03715299, -0.048978977, 0.07355...","[0.037736874, -0.27821398, 0.2481085, 0.019750...","[0.38141084, 0.6220578, -0.30403435, -0.665752...","[0.32039815, 0.6126691, 0.068923764, -0.711237...",0.62664282,1,0.62784225,0
83,566,今天早盤拉上去的時候\n 想不開\n 就跳海了\n 剩下的給別人賺\n 祝大家多空都賺大錢\...,今天還能拉漲停的話\n \n 那他真的是海賊王了\n \n 哈哈哈!!!!\n \n 不可能...,"When I pulled up this morning, I went up early...",If you can pull the daily limit today\n \n The...,"[0.22485298, 0.40419072, 0.046977628, -0.02726...","[0.11620488, 0.4570548, 0.08846752, -0.3904846...","[0.046648428, 0.84257126, 0.14790465, -0.71993...","[0.096996605, 0.68262774, 0.0058463304, -1.150...",0.34840301,1,0.31140193,0
84,568,"不知各位對於這隻股的看法如何?? 是在2010/11 Money 雜誌上看到推薦這隻股,看他...",近期外資回補了八萬多張，\n 看起來後勢可期^^,I wonder what the opinion of this stock ?? I s...,"Recently, foreign capital has replenished more...","[0.13683803, 0.052711, 0.11344689, 0.18187611,...","[0.126638, 0.1757161, 0.35074553, -0.001050096...","[0.089465216, 0.9457052, 0.14676356, -0.494651...","[0.3613182, 0.6352121, 0.22215028, -0.67816705...",0.82852507,1,0.84167242,0
85,570,BDI指數持續看漲，前10大航商組成三大聯盟，佔全球運力近八成，談價空間有利於航商，目前雞蛋...,我朋友於去年4月跟我對賭\n 說一年內鴻海的股價一定會落到40 (只要有碰到40就算)\n ...,The BDI index continues to rise. The top 10 na...,My friend gambled me in April last year\n It i...,"[0.195929, 0.4238964, 0.107287586, 0.13004434,...","[-0.15792839, 0.18903646, 0.36343715, 0.002221.

In [74]:
def compare_for_sort(txt1, txt2, emd_model, clf):
  txt1_emd = emd_model.encode([txt1])
  txt2_emd = emd_model.encode([txt2])
  pred = clf.predict(pd.concat([pd.DataFrame(txt1_emd), pd.DataFrame(txt2_emd)], axis = 1))[0]
  return pred

In [78]:
#Source: https://www.geeksforgeeks.org/python-program-for-bubble-sort/

# Python program for implementation of Bubble Sort
 
def bubbleSort(arr, emd_model, clf, task, lang):
    """
    emd_model: the embedding model... english or chine sbert model
    clf: the trained classifier
    task: MPP or ML
    lang: English or Chinese
    """
    n = len(arr)
    # optimize code, so if the array is already sorted, it doesn't need
    # to go through the entire process
    swapped = False
    # Traverse through all array elements
    for i in range(n-1):
        # range(n) also work but outer loop will
        # repeat one time more than needed.
        # Last i elements are already in place
        for j in range(0, n-i-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
          if lang == "English":  
            txt1 = unsupervised_test[unsupervised_test['index']==j]['post_rationale_1_2_english'].values[0]
            txt2 = unsupervised_test[unsupervised_test['index']==j+1]['post_rationale_1_2_english'].values[0]
          else:
            txt1 = unsupervised_test[unsupervised_test['index']==j]['post_rationale_1_2'].values[0]
            txt2 = unsupervised_test[unsupervised_test['index']==j+1]['post_rationale_1_2'].values[0]            
          pred = compare_for_sort(txt1, txt2, emd_model, clf)
          if task=="ML":
            if pred>0:
                swapped = True
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
          else:
            if pred<=0:
              swapped = True
              arr[j], arr[j + 1] = arr[j + 1], arr[j]
        if not swapped:
            # if we haven't needed to make a single swap, we
            # can just exit the main loop.
            return
 
 


In [ ]:
# Driver code to test above
arr = list(unsupervised_test['index'])

emd_model=model_chinese 
clf = clf
task = "ML"
lang = "English"
bubbleSort(arr, emd_model, clf, task, lang)
 
print("Sorted array is:")
for i in range(len(arr)):
    print("% d" % arr[i], end=" ")